# Matchmaking  Networks 🕸️

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
import pathlib
import sys

import graph_tool.all as gt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import powerlaw
import seaborn as sns

from matplotlib import colormaps as cm
from matplotlib.colors import Normalize
from scipy.stats import norm, gumbel_r
from tqdm import tqdm
import random

from IPython.display import display, HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats


# Set plot theme
sns.set_theme(style="whitegrid")

# Center figures on Juypter Notebook
display(HTML("<style>.output_png { display: flex; justify-content: center; }</style>"))

In [ ]:
YEAR = sys.argv[1] if len(sys.argv) > 1 else "2014"
YEAR = "2014-2015-2016"
ROOT_PATH = pathlib.Path("..")
DATA_PATH = ROOT_PATH / "data"
CSV_PATHS = sorted((DATA_PATH / "csv").glob(f"*201[4-6]*.csv"))
GT_PATH = DATA_PATH / "gt" / f"{YEAR}.gt"

assert len(CSV_PATHS) > 0

## Introduction


Given a pool of players (nodes), pair them **by skill level $X$** to ensure enjoyable games (links). 

In [ ]:
np.random.seed(42)

# param N_elos (f), dist of players elos
mu_f = 1500
o_f = 250
o2_f = o_f ** 2

players = {
    0: {"pos": (0, 0)},
    1: {"pos": (1, 4)},
    2: {"pos": (4, 3)},
    3: {"pos": (6, 1)},
    4: {"pos": (6, 6)},
    5: {"pos": (3, 0)},
}

# generate ELO by sampling from N_elos
elos = np.empty(len(players))
for i, player in players.items():
    elo = int(np.random.normal(mu_f, o_f))
    players[i]["elo"] = elo
    elos[i] = elo

# add color based on ELO
normalize = plt.Normalize(min(elos), max(elos))
cmap = cm["viridis"]
for i, player in players.items():
    players[i]["color"] = cmap(normalize(player["elo"]))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Draw the rectangle (pool)
offset = 1.5
x = min(p["pos"][0] for p in players.values()) - offset
y = min(p["pos"][1] for p in players.values()) - offset
w = max(p["pos"][0] for p in players.values()) + 2 * offset
h = max(p["pos"][1] for p in players.values()) + 2 * offset

rectangle1 = plt.Rectangle((x, y), w, h, fill=None, edgecolor='black')
rectangle2 = plt.Rectangle((x, y), w, h, fill=None, edgecolor='black')
ax1.add_patch(rectangle1)
ax2.add_patch(rectangle2)

ax1.set_xlim(x - .5, x + w + .5)
ax1.set_ylim(y - .5, y + h + .5)
ax1.axis('off')
ax1.set_aspect('equal')

ax2.set_xlim(x - .5, x + w + .5)
ax2.set_ylim(y - .5, y + h + .5)
ax2.axis('off')
ax2.set_aspect('equal')

# Plot the players as circles with their index on the first plot
for I, player in players.items():
    x, y = player["pos"]
    circle = plt.Circle((x, y), 0.5, facecolor='black', fill=True, edgecolor='black', linewidth=1.5)
    ax1.add_patch(circle)
    ax1.text(x, y, I, fontsize=15, ha='center', va='center', color='white')


# Plot the players as circles colored by ELO on the second plot
for I, player in players.items():
    x, y = player["pos"]
    circle = plt.Circle((x, y), 0.5, facecolor=player["color"], fill=True, edgecolor='black', linewidth=1.5)
    ax2.add_patch(circle)
    ax2.text(x, y + .8, player["elo"], fontsize=15, ha='center', va='center', color='black')

1. Start from $N$ players with associated rating (color) $x$ 

In [ ]:
eg = gt.Graph(directed=True)

vp_idx = eg.new_vertex_property("short")
vp_pos = eg.new_vertex_property("vector<double>")
vp_elo = eg.new_vertex_property("short")
vp_color = eg.new_vertex_property("vector<double>")
ep_num = eg.new_edge_property("int", val=0)

for i, player in players.items():
    v = eg.add_vertex()
    x, y = player["pos"]
    vp_idx[v] = i
    vp_pos[v] = (x, -y)
    vp_elo[v] = player["elo"]
    vp_color[v] = player["color"]

eg.vp["idx"] = vp_idx
eg.vp["pos"] = vp_pos
eg.vp["elo"] = vp_elo
eg.vp["color"] = vp_color
eg.ep["num"] = ep_num


gt.graph_draw(
    eg, 
    pos=eg.vp["pos"], 
    vertex_fill_color=eg.vp["color"],
    output_size=(350,350),
    vertex_size=48,
    vertex_pen_width=1.5,
    edge_pen_width=gt.prop_to_size(eg.ep["num"]),
    edge_end_marker="arrow",
);

2. Pairs two players. Players with similar rating have higher probability to be paired.

In [ ]:
def get_target(v, g):
    # params N_elo (g)
    mu_g = g.vp["elo"][v]
    o_g = 200
    o2_g = o_g ** 2

    # params N (fg)
    mu_fg = (mu_f * o2_g + mu_g * o2_f) / (o2_f + o2_g)
    o_fg = np.sqrt((o2_f * o2_g) / (o2_f + o2_g))
    o2_fg = o_fg ** 2
    
    # draw a sample form the N_fg distribution
    sample = np.random.normal(mu_fg, o_fg)
    
    elos[int(v)] = np.infty # avoid self loop
    u = g.vertex(abs(elos - sample).argmin())
    elos[int(v)] = g.vp["elo"][v] # restore elos arr
    return u

def add_edge(g):
    v = g.vertex(np.random.randint(0, len(g)))
    u = get_target(v, g)
    vu = g.edge(v, u)
    if vu is None:
        vu = g.add_edge(v, u, add_missing=False)
    g.ep["num"][vu] += 1

In [ ]:
add_edge(eg)

gt.graph_draw(
    eg, 
    pos=eg.vp["pos"], 
    vertex_fill_color=eg.vp["color"],
    output_size=(350,350),
    vertex_size=48,
    vertex_pen_width=1.5,
    edge_pen_width=3,
    edge_end_marker="arrow",
);

3. Repeat. Increase edge weight by one if the edge already exists.

In [ ]:
for _ in range(10_000):
    add_edge(eg)
    
gt.graph_draw(
    eg, 
    pos=eg.vp["pos"], 
    vertex_fill_color=eg.vp["color"],
    output_size=(350,350),
    vertex_size=48,
    vertex_pen_width=1.5,
    edge_pen_width=gt.prop_to_size(eg.ep["num"]),
);

Let’s focus on the player with highest rating (yellow node):

- Strong links with next strong players (green / blue nodes)
- Few links with low rated players (dark purple nodes)
- Links weights are not symmetric



In [ ]:
_eg = gt.Graph(eg) 

max_i = np.array([_eg.vp["elo"] for v in _eg.vertices()]).argmax()

for ij in [e for e in _eg.edges()]:
    i, j = ij.source(), ij.target()
    if i == max_i or j == max_i:
        continue
    _eg.remove_edge(ij)
    _eg.ep["num"][ij] = 0

gt.graph_draw(
    _eg, 
    #pos=_eg.vp["pos"], 
    vertex_fill_color=_eg.vp["color"],
    output_size=(350,350),
    vertex_size=48,
    vertex_pen_width=1.5,
    edge_pen_width=gt.prop_to_size(_eg.ep["num"]),
    edge_end_marker="arrow",
);

## Rating System

Many matchmaking algorithms rely on rating system to provide fair matches, i.e.


***a scalar value quantifies the player’s strength relative to other players.***

---

Matchmaking based on rating systems is employed in

- **Board Games**: Chess, Go, Backgammon, ...

- **Video Games**: League of Legends,  Dota 2, CS:GO, ...

- **Sports**: Tennis, Table Tennis, ...

---

I've studied the emerging matchmaking network on **chess games** dataset.

### Elo - Definition

Proposed by Arpad Elo, it infers rating from wins, losses, and draws against other players.

---
Let $E_A$ be the expected result for player $A$ with rating $x_A$ (same for player $B$)

$$
\begin{cases}
E_A \propto 10^{x_A / 400} \\
E_B \propto 10^{x_B / 400}
\end{cases}
\quad \textrm{such that} \quad
E_A + E_B = 1
$$

If the game result is $S_A$, $x_A'$ is the updated rating of player A:

$$
x_A' = x_A + K \cdot (S_A - E_A)
\quad \textrm{where} \quad
S_A = 
\begin{cases}
1   &\textrm{if $A$ wins}  \\
0.5 &\textrm{if $A$ draws} \\
0   &\textrm{if $A$ loses} \\
\end{cases}
$$

$K$ is a fixed parameter, usually in the range $[10, 40]$.


### Elo - Distribution

$$
E_A = \mathbb{P} (A \, \textrm{wins}) 
    = {\left[1 + e^{-\frac{\ln{10}}{400} (x_A - x_B )} \right]}^{-1} 
\quad
F_{\textrm{ Logistic}} (x \, | \, \mu, \, \beta ) 
= {\left[
1 + e^{-\frac{(x - \mu )}{\beta}} \right]}^{-1} 
$$

$$
X_A - X_B \sim 
\textrm{Logistic} \left(x_A - x_B \, | \, \mu=0, \, \beta=\frac{400}{\ln{10}} \right)
$$

---

*Property*

$$
\begin{cases}
X_A \sim \textrm{Gumbel }(x_A \, | \mu_{X_A}, \, \beta)\\
X_B \sim \textrm{Gumbel }(x_B \, | \mu_{X_B}, \, \beta)
\end{cases}
\quad \Rightarrow \quad
X_A - X_B \sim 
\textrm{Logistic} \left(x_A - x_B \, | \, \mu_{X_A} - \mu_{X_B}, \, \beta \right)
$$

---

$$
\Longrightarrow X \sim
\textrm{Gumbel}\left(x \, | \, \mu=\mu_{X}, \, \beta=\frac{400}{\ln{10}} \right) 
= \frac{1}{\beta} \exp{\left[\frac{x - \mu}{\beta} - \exp{\left(\frac{x - \mu}{\beta}\right)}\right]}
$$

In [ ]:
# Gumbel distribution
x = np.linspace(1000, 2800, 1000)
mu = 1500
beta = 400 / np.log(10)
y = gumbel_r.pdf(x, mu, beta)

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(14, 7))

# Plot the Gumbel distribution
sns.lineplot(x=x, y=y, color="tab:blue", linestyle="-", ax=ax, label=r'Gumbel $\left(\mu=1500, \, \beta=\frac{400}{\ln{10}}\right)$')
ax.set_xlabel('$x$', fontsize=15)
ax.set_yticklabels([])
ax.tick_params(axis='both', which='major', labelsize=13)
ax.legend(fontsize=15)
plt.show()

## Matchmaking Network - Chess Games 

- **Dataset**: games played on lichess.org between 2014 and 2016.
- **Rating System**: Glicko-2, an improvement over Elo.
- **Matchmaking**: pairs players with similar rating, $x \pm \Delta x$

### Players distribution

In [ ]:
def gen_df_games(csv_path: pathlib.Path, outliers_path:pathlib.Path) -> pd.DataFrame:
    cols = {
        "utc_date": "string",
        "utc_time": "string",
        "white": "string",
        "black": "string",
        "white_elo": "int",
        "black_elo": "int",
    }
    df = pd.read_csv(csv_path, dtype=cols, usecols=cols.keys())  # type: ignore

    # drrop outliers white or black
    with open(outliers_path, "r") as f:
        outliers = set(f.read().splitlines())
    df = df[~df["white"].isin(outliers) & ~df["black"].isin(outliers)]

    df["utc_datetime"] = pd.to_datetime(df["utc_date"] + " " + df["utc_time"])
    df.drop(columns=["utc_date", "utc_time"], inplace=True)
    df.sort_values(by="utc_datetime", inplace=True)
    df = df.reindex(
        columns=[
            "utc_datetime",
            "white",
            "black",
            "white_elo",
            "black_elo",
        ]
    )
    return df


def gen_df_players(df_games: pd.DataFrame) -> pd.DataFrame:
    df_melted = pd.melt(
        df_games,
        id_vars=["utc_datetime"],
        value_vars=["white", "black"],
        value_name="player",
        var_name="color",
    )
    df_melted["elo"] = pd.concat(
        [df_games["white_elo"], df_games["black_elo"]],
        ignore_index=True,
    )
    df_players = pd.DataFrame(
        {
            "elo": df_melted.groupby("player")["elo"].mean().astype(int),
            "num": df_melted.groupby("player")["elo"].count().astype(int),
        }
    )

    # > 20 games is the criteria used by Lichess in Arena Tournaments.
    # I select player > 50 games to avoid burn-out accounts.
    # Also the account with more than 4000 games per month are bot or are violating TOS.
    # df_players = df_players[(df_players["num"] > 50)]  # & (df_players["num"] < 4000)]
    df_players = df_players[(df_players["num"] > 20)]
    df_players.dropna(subset=["elo"], inplace=True)

    # assign a bin to each player based on their elo
    elo_min = df_players["elo"].min() // 50 * 50
    elo_max = df_players["elo"].max() // 50 * 50
    bins = np.arange(elo_min, elo_max + 50, 50, dtype=int)
    df_players["bin"] = pd.cut(df_players["elo"], bins=bins, labels=bins[:-1])
    df_players.dropna(subset=["bin"], inplace=True)
    df_players["bin"] = df_players["bin"].astype(int)

    # This is important because it make the creation of the assorativity matrix easier
    df_players.sort_values(by="elo", ascending=True, inplace=True)

    # assign a int to each player to be used as vertex
    df_players["v"] = np.arange(len(df_players), dtype=int)
    df_players["v_bin"] = df_players["bin"].map({bin: i for i, bin in enumerate(bins)})
    return df_players


def generate_dfs(
        csv_paths: list[pathlib.Path],
        outliers_path:pathlib.Path = DATA_PATH / "outliers.txt",
    ) -> tuple[pd.DataFrame, pd.DataFrame]:
    df_games = tqdm((gen_df_games(path, outliers_path) for path in csv_paths), total=len(csv_paths))
    df_games = pd.concat(df_games, ignore_index=True)
    df_players = gen_df_players(df_games)

    df_games = df_games[
        df_games["white"].isin(df_players.index)
        & df_games["black"].isin(df_players.index)
    ]

    # add vertex id to games
    df_games["v_white"] = df_games["white"].map(df_players["v"])
    df_games["v_black"] = df_games["black"].map(df_players["v"])
    df_games["v_bin_white"] = df_games["white"].map(df_players["v_bin"])
    df_games["v_bin_black"] = df_games["black"].map(df_players["v_bin"])
    
    return df_games, df_players


df_games, df_players = generate_dfs(CSV_PATHS)

In [ ]:
def plot_players_distribution(df_players: pd.DataFrame):
    # histplot
    fig, ax = plt.subplots(figsize=(14, 6))
    sns.histplot(
        df_players,
        x="elo",
        bins=df_players["bin"].unique(),
        stat="density",
        ax=ax,
    )
    
    # Fit a Gumbel distribution
    params = gumbel_r.fit(df_players["elo"])
    loc, scale = params
    x = np.linspace(df_players["elo"].min(), df_players["elo"].max(), 100)
    pdf = gumbel_r.pdf(x, loc, scale)

    # Plot the Gumbel distribution
    sns.lineplot(x=x, y=pdf, color="black", linestyle="--", ax=ax, label="Gumbel Fit")

    # Fit a normal distribution
    elo_mean = df_players["elo"].mean()
    elo_std = df_players["elo"].std()
    x = np.linspace(df_players["elo"].min(), df_players["elo"].max(), 100)
    pdf = norm.pdf(x, elo_mean, elo_std)

    # Plot the normal distribution
    sns.lineplot(x=x, y=pdf, color="black", linestyle="-", ax=ax, label="Normal Fit")

    # Color based on elo
    norm_color = plt.Normalize(df_players["bin"].min(), df_players["bin"].max())
    for patch in ax.patches:
        bar_center = patch.get_x() + patch.get_width() / 2
        patch.set_color(cm["viridis"](norm_color(bar_center)))
        patch.set_edgecolor("white")
        

    ax.set_ylabel('Density', fontsize=15)
    ax.set_xlabel('$x$', fontsize=15)
    ax.set_yticklabels([])
    ax.tick_params(axis='both', which='major', labelsize=13)
    ax.legend(fontsize=15)


plot_players_distribution(df_players)

### Games distribution

In [ ]:
def plot_games_distribution(df_players: pd.DataFrame):
    fig, ax = plt.subplots(figsize=(14, 6))

    sns.histplot(
        df_players,
        x="num",
        hue="bin",
        stat="count",
        multiple="stack",
        palette="viridis",
        edgecolor=None,
        bins=50,
        ax=ax,
        legend=False,
    )
    ax.set_yscale("log")

    ax.set_ylabel('# Players', fontsize=15)
    ax.set_xlabel('# Games', fontsize=15)
    ax.tick_params(axis='both', which='major', labelsize=13)


plot_games_distribution(df_players)

### Graph



In [ ]:
def gen_graph(df_games: pd.DataFrame, df_players: pd.DataFrame) -> gt.Graph:
    if GT_PATH.exists():
        with open(GT_PATH, "rb") as f:
            return gt.load_graph(f)

    g = gt.Graph(directed=True)
    vp_elo = g.new_vp("short", val=1500)  # elo is initialized to 1500
    ep_num = g.new_ep("short")  # number of games between players

    # add edges and edge properties
    white_groups = tqdm(df_games.groupby("v_white"), total=len(df_players))
    for v_white, white_games in white_groups:
        for v_black, black_games in white_games.groupby("v_black"):
            e = g.add_edge(v_white, v_black)
            ep_num[e] = len(black_games)

    # add vertex properties
    for player, row in df_players.iterrows():
        v = g.vertex(row.v)
        vp_elo[v] = row.elo

    g.ep["num"] = ep_num
    g.vp["elo"] = vp_elo
    
    # This help to exclude games manually arranged by players
    g_connected = gt.extract_largest_component(g, prune=False)

    with open(GT_PATH, "wb") as f:
        g_connected.save(f)

    return g_connected

g = gen_graph(df_games, df_players)
g

- **Nodes** 
    - represent a player
    - property: $x$
    - $N = 172,195$

- **Links**
    - represent games played between two players
    - directed: player with white pieces is the source
    - weight: number of games
    - $L = 30,081,979$

In [ ]:
assort, var_assort = gt.scalar_assortativity(g, g.vp["elo"])
print(f"Scalar assortativity:\t {assort:.4f} ± {var_assort:.4f}")

- **$x$-assortativity**
    - expected high positive value
    - $0.7612 \pm 0.0001$

In [ ]:
def plot_assortativity_matrix_g(g: gt.Graph):
    adj = gt.adjacency(g)
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.spy(adj, markersize=0.01, origin="lower", marker=".")

    # Set the ticks and labels to the sorted ELOs
    tick_indices = np.linspace(
        start=0,
        stop=adj.shape[0] - 10,  # a little bit of padding
        num=8,  # number of ticks
        endpoint=True,
        dtype=int,
    )
    tick_labels = [g.vp["elo"][idx] for idx in tick_indices]

    ax.set_xticks(tick_indices[1:])
    ax.set_yticks(tick_indices[:])
    ax.set_xticklabels(tick_labels[1:], rotation=45)
    ax.set_yticklabels(tick_labels[:])
    ax.tick_params(axis='both', which='major', labelsize=13)

    
#  This is to hard to plot with SVG backend
with plt.rc_context({'savefig.format': 'png'}):
    set_matplotlib_formats('retina')
    plot_assortativity_matrix_g(g)

- **Degree distribution**
    - Does not follow power-law
    - Physical limit (games per month) & Structural limit (nodes with extreme $x$)

In [ ]:
k_in = np.fromiter((v.in_degree() for v in g.vertices()), dtype=int)
k_out = np.fromiter((v.out_degree() for v in g.vertices()), dtype=int)
k_tot = k_in + k_out

k_out = k_out[k_out > 10]
k_tot = k_tot[k_tot > 20]


bin_edges = np.logspace(np.log10(min(k_tot)), np.log10(max(k_tot)), num=30)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Create a figure and axis object
fig, ax = plt.subplots(figsize=(9, 4.5))

sns.scatterplot(x=bin_centers, y=np.histogram(k_tot, bins=bin_edges)[0] / sum(k_tot), marker='s', ax=ax, label=r"$P_{k^{\text{tot}}}$", s=100)
sns.scatterplot(x=bin_centers, y=np.histogram(k_out, bins=bin_edges)[0] / sum(k_out), marker='o', ax=ax, label=r"$P_{k^{\text{out}}}$", s=100)

ax.set_xlabel(r"$k$",  fontsize=15)
ax.set_ylabel(r"$P_k$",fontsize=15)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=13)
ax.legend(fontsize=15)
plt.show()

In [ ]:
dist_counts, dist_bins = gt.distance_histogram(g, samples=2000, float_count=False)
dist_avg = sum(dist_counts * dist_bins[:-1]) / sum(dist_counts)
dist_max = gt.pseudo_diameter(g, source=0)

print("avg dist:", dist_avg)
print("max dist:", dist_max)

- **Distances**
    - computed over 2000 sampled nodes: $\langle d \rangle = 2.90 $     
    - pseudo-diameter: $d_{\max} = 8$

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4.5))

sns.barplot(x=dist_bins[1:-1], y=dist_counts[1:] / sum(dist_counts), ax=ax)
ax.set_xlabel(r"$d$",  fontsize=15)
ax.set_ylabel(r"$P_d$",fontsize=15)
ax.set_yscale('log')
ax.tick_params(axis='both', which='major', labelsize=13)
plt.show()

### Reduced Graph


In [ ]:
def gen_reduced_graph(df_games: pd.DataFrame, df_players: pd.DataFrame):
    num_bins = df_players["v_bin"].max() + 1
    rg = gt.complete_graph(num_bins, self_loops=True, directed=True)
    vp_elo = rg.new_vp("short")  # elo of the bin
    vp_num = rg.new_vp("long")  # number of player in each bin
    ep_num = rg.new_ep("long", val=0)  # number of games between bins of players

    # add vertex properties
    for v, (bin, bin_players) in enumerate(df_players.groupby("bin")):
        vp_elo[v] = bin
        vp_num[v] = len(bin_players)

    # add edges and edge properties
    v_bin_white_group = tqdm(df_games.groupby("v_bin_white"), total=num_bins)
    for v_bin_white, white_games in v_bin_white_group:
        for v_bin_black, black_games in white_games.groupby("v_bin_black"):
            e = rg.edge(v_bin_white, v_bin_black)
            ep_num[e] += len(black_games)

    rg.ep["num"] = ep_num
    rg.vp["elo"] = vp_elo
    rg.vp["num"] = vp_num
    return rg

rg = gen_reduced_graph(df_games, df_players)

print("Reudced Graph")
print("=" * 40)
print("Number of vertices:\t", rg.num_vertices())
print("Number of edges:\t", rg.num_edges())

Group players with same $x$-bin into single node.

- **Nodes** 
    - represent a bin of size 50
    - properties: $x$-bin, # players
    - $N = 39$

- **Links**
    - represent games between bins of players
    - directed: players with white pieces are the source
    - weight: number of games
    - self-loops: players of the same bin can play against each other
    - $L = N^2 = 1521$

In [ ]:
def draw_reduced_graph(df_players: pd.DataFrame, rg: gt.Graph):
    vp_pos = rg.new_vp("vector<double>")
    vp_color = rg.new_vp("vector<double>")
    ep_color = rg.new_ep("vector<double>")
    ep_width = rg.new_ep("vector<double>")

    cmap = cm["viridis"]
    norm = Normalize(vmin=min(rg.vp["elo"].a), vmax=max(rg.vp["elo"].a))
    step = 2 * np.pi / rg.num_vertices()
    for i, v in enumerate(rg.vertices()):
        vp_color[v] = cmap(norm(rg.vp["elo"][v]))
        vp_pos[v] = (np.cos(i * step), np.sin(i * step))

    vp_size = gt.prop_to_size(rg.vp["num"], mi=10, ma=35, log=False, power=0.5)

    cmap = cm["plasma"]
    norm = Normalize(vmin=min(rg.ep["num"].a), vmax=max(rg.ep["num"].a))
    for e in rg.edges():
        ep_color[e] = cmap(norm(rg.ep["num"][e]))
    ep_width = gt.prop_to_size(rg.ep["num"], mi=0.01, ma=5, log=False, power=0.5)

    gt.graph_draw(
        rg,
        ## Vertices
        pos=vp_pos,
        output_size=(550,550),
        vertex_fill_color=vp_color,
        vertex_size=vp_size,
        ## Edges
        # edge_color=ep_color,
        edge_pen_width=ep_width,
        edge_end_marker="none",
    )
    
draw_reduced_graph(df_players, rg)

In [ ]:
assort, var_assort = gt.scalar_assortativity(rg, rg.vp["elo"], eweight=rg.ep["num"])
print(f"Assort. (eweight):\t {assort:.2f} ± {var_assort:.2f}")

- **weighted $x$-assortativity**: $0.78 \pm 0.33$

In [ ]:
def plot_assortativity_matrix_rg(rg: gt.Graph):
    ep_weight = gt.prop_to_size(rg.ep["num"], mi=0.01, ma=5, log=False, power=0.5)
    mat = np.zeros((rg.num_vertices(), rg.num_vertices()))
    for e in rg.edges():
        i = int(e.source())
        j = int(e.target())
        mat[i, j] += ep_weight[e]

    # Normalize the matrix across each row
    mat /= mat.sum(axis=1, keepdims=True)

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(mat, cmap="magma", origin="lower")

    tick_indices = np.linspace(
        start=0,
        stop=mat.shape[0] - 1,
        num=8,
        endpoint=True,
        dtype=int,
    )
    tick_labels = [rg.vp["elo"][idx] for idx in tick_indices]
    ax.set_xticks(tick_indices[1:])
    ax.set_yticks(tick_indices[:])
    ax.set_xticklabels(tick_labels[1:], rotation=45)
    ax.set_yticklabels(tick_labels[:])
    ax.set_ylabel(r"White $x$-bins", fontsize=15)
    ax.set_xlabel(r"Black $x$-bins", fontsize=15)
    ax.tick_params(axis='both', which='major', labelsize=13)

    # Save the figure
    fig.tight_layout()

plot_assortativity_matrix_rg(rg)

## Matchmaking Network - Simple Model

### Recipe

---
1. Start from fix set of $N$ nodes with rating $X \sim \mathcal{N}(\mu_{x}, \sigma_{x})$
---
2. Select a random node $s$ as source
---
3. Sample from distribution $ \propto \mathcal{N} (\mu_{x}, \sigma_{x}) \cdot \mathcal{N} (x_s, \alpha)$ a rating $x_t$
---
4. Add the edge $s \rightarrow t$ where $t :=$ argmin $|x_v - x_{t}|$ (with $v \neq s$)
---

*... and repeat 2. to 4.*

$$
\mathcal{N} (\mu_t, \beta \,) =
\mathcal{N} (\mu_{x}, \sigma_{x}) \cdot \mathcal{N} (x_s, \alpha)
\quad \text{where} \quad
\mu_{t} := \frac{\mu_x \alpha^2 + x_s \sigma^2_x}{\sigma^2_x + \alpha^2}
\quad
\beta := \sqrt{\frac{\sigma^2_x \alpha^2}{\sigma^2_x + \alpha^2}}
$$

---

$$
\mathcal{N}_x \equiv
\mathcal{N} (\mu_{x}, \sigma_{x}) \,
\begin{cases}
\mu_{x} = 1500 \\
\sigma_{x} = 250
\end{cases}
\quad
\mathcal{N}_s \equiv
\mathcal{N} (x_s, \alpha) \,
\begin{cases}
x_s = 2300 \\
\alpha = 150
\end{cases}
\quad \Longrightarrow \quad
\mathcal{N}_t
\begin{cases}
\mu_t \approx 2088 \\
\beta \approx 129
\end{cases}
$$

In [ ]:
# params N_x
mu_x = 1500
o_x = 250
o2_x = o_x ** 2

# params N_s
x_s = 2300
a = 150
a2 = a ** 2

# params N_t
mu_t = (mu_x * a2 + x_s * o2_x) / (o2_x + a2)
b = np.sqrt((o2_x * a2) / (o2_x + a2))

# distributions
x = np.linspace(500, 3000, 1000)
N_x = (1/(o_x * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu_x)/o_x )**2)
N_s = (1/(a   * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - x_s) /a   )**2)
N_t = (1/(b   * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu_t) /b )**2)

fig, ax = plt.subplots(figsize=(9, 3.5))
ax.plot(x, N_x, label=r"$\mathcal{N}_x$", linestyle="solid",  linewidth=2.5)
ax.plot(x, N_s, label=r"$\mathcal{N}_s$", linestyle="dashed", linewidth=2.5)
ax.plot(x, N_t, label=r"$\mathcal{N}_t$", linestyle="dotted", linewidth=2.5)
ax.set_xlabel('$x$', fontsize=15)
ax.set_yticklabels([])
ax.tick_params(axis='both', which='major', labelsize=13)
ax.legend(fontsize=15)
plt.show()

In [ ]:
def matchmaking_network(N, L, mu_x, o_x, alpha, mix=True):
    np.random.seed(1)

    G = gt.Graph(N)
    xs = np.random.normal(mu_x, o_x, size=N).astype(int)
    xs = np.sort(xs) # this make it easier to draw assort. matrix
    ep_w = G.new_edge_property("int", val=0)
    vp_x = G.new_vertex_property("int", vals=xs)
    
    def sample_x_t(x_s):
        o2_x, a2 = o_x * o_x, alpha * alpha
        mu_t = (mu_x * a2 + x_s * o2_x) / (o2_x + a2)
        b = np.sqrt((o2_x * a2) / (o2_x + a2))
        return np.random.normal(mu_t, b)

    def add_edge(s, x_t):
        x_s = xs[s]
        xs[s] = -1
        diff = np.abs(xs - x_t)
        min_indices = np.where(diff == diff.min())[0]
        t = np.random.choice(min_indices)
        xs[s] = x_s
        st = G.edge(s, t)
        if st is None:
            st = G.add_edge(s, t)
        ep_w[st] += 1

    # ... repeat 2. to 4.
    for _ in range(L):
        s = np.random.randint(0, len(G))
        x_t = sample_x_t(xs[s])
        add_edge(s, x_t)
    
    G.ep["w"] = ep_w
    G.vp["x"] = vp_x
    return G


def draw_matchmaking_network(G, p=2, C=0.2):
    np.random.seed(3)
    vp_x = G.vp["x"]
    ep_w = G.ep["w"]
    
    # initial position
    init_pos = np.stack([vp_x.a, 100 * np.random.rand(len(vp_x.a))], axis=1)
    vp_init_pos = G.new_vertex_property("vector<float>", vals=init_pos)
    vp_pos = gt.sfdp_layout(G, eweight=ep_w)#, pos=vp_init_pos, C=C, p=p)

    # vertex fill color
    norm_color = plt.Normalize(min(vp_x), max(vp_x))
    x_to_color = lambda x: cm["viridis"](norm_color(x))
    xs_to_colors = np.vectorize(x_to_color, otypes=[np.ndarray])
    vp_color = G.new_vertex_property("vector<float>", vals=xs_to_colors(vp_x))
    
    gt.graph_draw(
        G, 
        output_size=(600, 500),
        pos=vp_pos,
        vertex_fill_color=vp_color,
        edge_pen_width=gt.prop_to_size(ep_w),
    )

**Example:**$\quad N = 300 \quad L = 30,000 \quad \mathcal{N}(\mu_x = 1500, \sigma_x = 400) \quad \alpha = 200 $

In [ ]:
G = matchmaking_network(N=300, L=15_000, mu_x=1500, o_x=400, alpha=200)
draw_matchmaking_network(G, p=2.8, C=30)

In [ ]:
assort, var_assort = gt.scalar_assortativity(G, G.vp["x"], eweight=G.ep["w"])
print(f"Assort. (eweight):\t {assort:.2f} ± {var_assort:.2f}")

- **weighted $x$-assortativity**: $0.86 \pm 0.04$

In [ ]:
def plot_assortativity_matrix_G(g: gt.Graph):
    adj = gt.adjacency(g)
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.spy(adj, markersize=1, origin="lower", marker=".")

    # Set the ticks and labels to the sorted ELOs
    tick_indices = np.linspace(
        start=0,
        stop=adj.shape[0] - 10,  # a little bit of padding
        num=8,  # number of ticks
        endpoint=True,
        dtype=int,
    )
    tick_labels = [g.vp["x"][idx] for idx in tick_indices]

    ax.set_xticks(tick_indices[1:])
    ax.set_yticks(tick_indices[:])
    ax.set_xticklabels(tick_labels[1:], rotation=45)
    ax.set_yticklabels(tick_labels[:])
    ax.tick_params(axis='both', which='major', labelsize=13)

    
#  This is to hard to plot with SVG backend
with plt.rc_context({'savefig.format': 'png'}):
    set_matplotlib_formats('retina')
    plot_assortativity_matrix_G(G)

## References

[1] Elo, Arpad E. (August 1967). "The Proposed USCF Rating System, Its Development, Theory, and Applications". Chess Life. XXII (8): 242–247. [PDF](https://uscf1-nyc1.aodhosting.com/CL-AND-CR-ALL/CL-ALL/1967/1967_08.pdf#page=26)

[2] Arthur, Berg. (September 2020). "Statistical Analysis of the Elo Rating System in Chess". [LINK](https://chance.amstat.org/2020/09/chess/)

[3] Lichess.org, "Lichess Database." Accessed June 10, 2024. [LINK](https://database.lichess.org)

[4] Barabási, Albert-László. (2016). "Network Science". Cambridge University Press. [LINK](http://barabasi.com/networksciencebook/)

[5] Peixoto, Tiago P. (2014). "The graph-tool Python library". [LINK](https://graph-tool.skewed.de/)

[6] P.A. Bromiley (November 2003), "Product and Convolution of Gaussian Distribution". Internal report. [PDF](https://web.archive.org/web/20130517221128/http://www.tina-vision.net/docs/memos/2003-003.pdf)